# Constructing Support Vector Machine (SVM) regression model for OER adsorption energies from Seoin's dataset
---


### Import Modules

In [1]:
import os
print(os.getcwd())
import sys
import time; ti = time.time()

import pickle

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

import plotly.graph_objects as go

# #########################################################
from methods import (
    get_df_features_targets,
    get_df_features_targets_seoin,
    )

from methods_models import ModelAgent, GP_Regression

from proj_data import adsorbates
from proj_data import layout_shared
from proj_data import scatter_marker_props

/home/raulf2012/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/workflow/model_building/SVR/seoin_data
RegressionModel_2 will eventually replace  RegressionModel_1


In [2]:
from methods import isnotebook    
isnotebook_i = isnotebook()
if isnotebook_i:
    from tqdm.notebook import tqdm
    verbose = True
    show_plot = True
else:
    from tqdm import tqdm
    verbose = False
    show_plot = False

In [3]:
root_dir = os.path.join(
    os.environ["PROJ_irox_oer"],
    "workflow/model_building/linear_models/my_data")

### Script Inputs

In [4]:
# target_ads_i = "oh"

# feature_ads_i = "o"

# use_seoin_data = False

# if use_seoin_data:
#     feature_ads_i = "o"

In [5]:
quick_easy_settings = False
if quick_easy_settings:
    k_fold_partition_size = 170
    do_every_nth_pca_comp = 8
else:
    k_fold_partition_size = 10
    do_every_nth_pca_comp = 1

In [6]:
# # TEMP
# print(111 * "TEMP | ")
# do_every_nth_pca_comp = 9

### Read Data

In [7]:
# #########################################################
df_features_targets = get_df_features_targets()

# #########################################################
df_seoin = get_df_features_targets_seoin()

### Combine mine and Seoin's data

In [8]:
# if use_seoin_data:
#     # Replace multiindex with index of tuples so that my data and Seoin's data can be combined
#     indices = df_features_targets.index.tolist()
#     df_features_targets.index = indices

#     indices = df_seoin.index.tolist()
#     df_seoin.index = indices

#     # Remove columns that aren't shared by my and Seoin's data
#     cols_0 =df_features_targets.columns.tolist()
#     cols_1 = df_seoin.columns.tolist()

#     cols_comb = cols_0 + cols_1

#     cols_comb_unique = []
#     for col_i in cols_comb:
#         if col_i not in cols_comb_unique:
#             cols_comb_unique.append(col_i)

#     shared_cols = []
#     for col_i in cols_comb_unique:
#         if col_i in df_features_targets.columns and col_i in df_seoin.columns:
#             shared_cols.append(col_i)

#     # Combine data
#     df_data = pd.concat([
#         df_features_targets[shared_cols],
#         df_seoin[shared_cols],
#         ], axis=0)
# else:
#     df_data = df_features_targets

In [9]:
# # TEMP
# print(222 * "TEMP | ")

# df_data = df_data[df_data.data.stoich == "AB3"]

In [10]:
df_data = df_seoin

In [11]:
df_data = df_data[[
#     ('targets', 'g_o', ''),
    ('targets', 'g_oh', ''),
    ('data', 'stoich', ''),

    ('features', 'bulk_oxid_state', ''),
    ('features', 'dH_bulk', ''),
    ('features', 'effective_ox_state', ''),
    ('features', 'volume_pa', ''),
    ('features', 'o', 'active_o_metal_dist'),
    # ('features', 'o', 'angle_O_Ir_surf_norm'),
    ('features', 'o', 'ir_o_mean'),
    ('features', 'o', 'ir_o_std'),
    ('features', 'o', 'octa_vol')

    ]]

df_data

targets   data  \
                                                      g_oh stoich   
                                                                    
crystal facet coverage   termination active_site                    
amm2    012   OH_covered 0           0            1.120188    AB3   
        100   O_covered  0           0            0.700087    AB3   
        110   O_covered  0           0            1.307412    AB3   
                                     1            0.585981    AB3   
                                     2            1.099869    AB3   
...                                                    ...    ...   
rutile  120   O_covered  1           0            0.250262    AB2   
                                     1            0.008432    AB2   
        121   O_covered  0           0            0.376834    AB2   
                                     1            1.097317    AB2   
                                     2            0.830386    AB2   

                                                        features            \
                                                 bulk_oxid_state   dH_bulk   
                                                                             
crystal facet coverage   termination active_site                             
amm2    012   OH_covered 0           0                         6 -0.532592   
        100   O_covered  0           0                         6 -0.532592   
        110   O_covered  0           0                         6 -0.532592   
                                     1                         6 -0.532592   
                                     2                         6 -0.532592   
...                                                          ...       ...   
rutile  120   O_covered  1           0                         4 -0.834534   
                                     1                         4 -0.834534   
        121   O_covered  0           0                         4 -0.834534   
                                     1                         4 -0.834534   
                                     2                         4 -0.834534   

                                                                     \
                                                 effective_ox_state   
                                                                      
crystal facet coverage   termination active_site                      
amm2    012   OH_covered 0           0                     8.000000   
        100   O_covered  0           0                     7.000000   
        110   O_covered  0           0                     8.000000   
                                     1                     7.000000   
                                     2                     8.000000   
...                                                             ...   
rutile  120   O_covered  1           0                     6.000000   
                                     1                     5.333333   
        121   O_covered  0           0                     6.333333   
                                     1                     7.000000   
                                     2                     7.000000   

                                                             \
                                                  volume_pa   
                                                              
crystal facet coverage   termination active_site              
amm2    012   OH_covered 0           0            12.743355   
        100   O_covered  0           0            12.743355   
        110   O_covered  0           0            12.743355   
                                     1            12.743355   
                                     2            12.743355   
...                                                     ...   
rutile  120   O_covered  1           0            10.954316   
                                     1            10.954316   
        121   O_covere

In [12]:
from methods_models import SVR_Regression

SVR_R = SVR_Regression()

In [13]:
data_dict_list = []
num_feat_cols = df_data.features.shape[1]
# for num_pca_i in range(1, num_feat_cols + 1, do_every_nth_pca_comp):
for num_pca_i in range(4, num_feat_cols + 1, do_every_nth_pca_comp):

    if verbose:
        print("")
        print(40 * "*")
        print(num_pca_i)

    MA = ModelAgent(
        df_features_targets=df_data,
        Regression=SVR_R,
        Regression_class=SVR_Regression,
        use_pca=True,
        num_pca=num_pca_i,
        adsorbates=adsorbates,
        stand_targets=False,  # True was giving much worse errors, keep False
        )

    MA.run_kfold_cv_workflow(
        k_fold_partition_size=k_fold_partition_size,
        )

    if MA.can_run:
        if verbose:
            print("MAE:", np.round(MA.mae, 4))
            print("MA.r2:", np.round(MA.r2, 4))
            print("MAE (in_fold):", np.round(MA.mae_infold, 4))

    data_dict_i = dict()
    data_dict_i["num_pca"] = num_pca_i
    data_dict_i["MAE"] = MA.mae
    data_dict_i["ModelAgent"] = MA
    data_dict_list.append(data_dict_i)

df_models = pd.DataFrame(data_dict_list)
df_models = df_models.set_index("num_pca")




# #########################################################
# Finding best performing model
row_models_i = df_models.sort_values("MAE").iloc[0]

MA_best = row_models_i.ModelAgent

print(4 * "\n")
if verbose:
    print(
        row_models_i.name,
        " PCA components are ideal with an MAE of ",
        np.round(
        row_models_i.MAE,
            4),
        sep="")


****************************************
4
MAE: 0.1411
MA.r2: 0.6738
MAE (in_fold): 0.1034

****************************************
5
MAE: 0.1422
MA.r2: 0.6485
MAE (in_fold): 0.1002

****************************************
6
MAE: 0.1387
MA.r2: 0.664
MAE (in_fold): 0.0887

****************************************
7
MAE: 0.1335
MA.r2: 0.6664
MAE (in_fold): 0.086

****************************************
8
MAE: 0.1295
MA.r2: 0.6885
MAE (in_fold): 0.0802





8 PCA components are ideal with an MAE of 0.1295


In [14]:
# 6 PCA components are ideal with an MAE of 0.1339

In [15]:
assert False

AssertionError: 

In [ ]:
# 9 PCA components are ideal with an MAE of 0.1282



In [ ]:
from methods_models import ModelAgent_Plotter

MA_Plot = ModelAgent_Plotter(
    ModelAgent=MA_best,
    layout_shared=layout_shared,
    )

MA_Plot.plot_residuals()
MA_Plot.plot_parity()
MA_Plot.plot_parity_infold()

# # Uncomment to run pca analysis on in-fold regression
# MA.run_pca_analysis()

In [ ]:
fig = MA_Plot.plot_residuals__PLT
if show_plot:
    fig.show()

In [ ]:
fig = MA_Plot.plot_parity__PLT
if show_plot:
    fig.show()

In [ ]:
fig = MA_Plot.plot_parity_infold__PLT
if show_plot:
    fig.show()

In [ ]:
from methods_models import plot_mae_vs_pca
plot_mae_vs_pca(
    df_models=df_models,
    layout_shared=layout_shared,
    scatter_marker_props=scatter_marker_props,
    )

### Save Data

In [ ]:
# Deleting cinv matrix of GP model to save disk space

for num_pca, row_i in df_models.iterrows():
    MA = row_i.ModelAgent
    # MA.cleanup_for_pickle()

In [ ]:
data_dict_out = {
    "df_models": df_models,
    "ModelAgent_Plot": MA_Plot,
    }

In [ ]:
# Pickling data ###########################################
directory = os.path.join(root_dir, "out_data")
print(directory)
if not os.path.exists(directory): os.makedirs(directory)
with open(os.path.join(directory, "modelling_data.pickle"), "wb") as fle:
    pickle.dump(data_dict_out, fle)
# #########################################################

In [ ]:
# #########################################################
print(20 * "# # ")
print("All done!")
print("Run time:", np.round((time.time() - ti) / 60, 3), "min")
print("model__mine_GP.ipynb")
print(20 * "# # ")
# #########################################################

In [ ]:
# kdict = [
#     {
#         "type": "gaussian",
#         "dimension": "single",
#         "width": 1.8,
#         "scaling": 0.5,
#         "scaling_bounds": ((0.0001, 10.),),
#         }
#     ]

# GP_R = GP_Regression(
#     kernel_list=kdict,
#     regularization=0.01,
#     optimize_hyperparameters=True,
#     scale_data=False,
#     )

In [ ]:
# assert False